In [ ]:
import pymysql
import pprint
import pandas as pd
import sys,os
import passwords

mysqlip = passwords.mysqlip
mysqlpassword = passwords.mysqlpassword
mysqlusername = passwords.mysqlusername
mysqldb = passwords.mysqldb
mysqldb = 'joco2022'

pp = pprint.PrettyPrinter(indent=4)

In [ ]:
def sendSQLCommand(sql):
    try:
        cursor = connection.cursor()
        cursor.execute(sql)
        dockets = cursor.fetchall()
        cursor.close()
        return dockets
    except Exception as e:
        print(e)
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(exc_type, fname, exc_tb.tb_lineno)
        return None



In [ ]:
sql = '''
SET sql_mode = 'STRICT_TRANS_TABLES,NO_ZERO_IN_DATE,NO_ZERO_DATE,ERROR_FOR_DIVISION_BY_ZERO,NO_ENGINE_SUBSTITUTION';
'''

connection = pymysql.connect(host=mysqlip,
                             user=mysqlusername,
                             password=mysqlpassword,
                             db=mysqldb,
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
answer = sendSQLCommand(sql)

In [ ]:
#Total Cases in Each County
sql = "SELECT Court as County,COUNT(*) as Cases FROM kansas2022.case_information GROUP BY County UNION SELECT 'Johnson County' as County,COUNT(*) as Cases FROM joco2022.caseinfo"
answer = sendSQLCommand(sql)
print(answer)
for ta in answer:
    ta['County'] = ta['County'].replace("County","")
df = pd.DataFrame(answer)
display(df.sort_values(by=['Cases'],ascending=False).head(10))
title = 'Top Ten Jurisdictions with the Most Criminal Cases'
temp = df.sort_values(by=['Cases'],ascending=False).head(10).plot(kind="bar",x='County',ylabel="Amount of Cases",title=title)

temp.figure.savefig("Graphs/" + title +".png",bbox_inches="tight")

In [ ]:
#Total Cases in Joco and Non Joco
sql = "SELECT 'Non-Johnson County' as County, COUNT(*) as Cases FROM kansas2022.case_information UNION SELECT 'Johnson County' as County,COUNT(*) FROM joco2022.caseinfo"
answer = sendSQLCommand(sql)
df = pd.DataFrame(answer)
display(df.sort_values(by=['Cases'],ascending=False).head(5))
title = 'Criminal Case Count Aggregating Courts Outside of Johnson County'
temp = df.sort_values(by=['Cases'],ascending=False).head(5).plot(kind="bar",x='County',ylabel="Amount of Cases",title=title)
temp.figure.savefig("Graphs/" + title + ".png",bbox_inches="tight")

In [ ]:
#Johnson County Cases By Year
sql = '''
SELECT YEAR(date) as 'Filing Date',COUNT(*) as Cases FROM (SELECT date from joco2022.charges WHERE date is not NULL GROUP BY casenumber) as temp GROUP BY YEAR(date) ORDER BY YEAR(date) desc
'''
answer = sendSQLCommand(sql)
newanswer = []
for temp in answer:
    newanswer.append({'Filing Date':str(temp['Filing Date']),'Cases':temp['Cases']})
df = pd.DataFrame(newanswer)
df['Filing Date'] = df['Filing Date'].astype(int)
df = df.loc[(df["Filing Date"] >= 1991)]
display(df.sort_values('Filing Date',ascending=True).head(30))
title = 'Criminal Cases in Johnson County By Filing Date'
temp = df.sort_values(by=['Filing Date'],ascending=True).head(30).plot(kind="bar",x='Filing Date',xlabel = 'Filing Year',ylabel="Amount of Cases",title=title)
temp.figure.savefig("Graphs/" + title +".png",bbox_inches="tight")

# Non Johnson County Cases By Year
sql = '''
SELECT COUNT(*) as Cases,YEAR(`File Date`) as 'Filing Date' FROM kansas2022.case_information GROUP BY YEAR(`File Date`) ORDER by YEAR(`File Date`) DESC
'''
answer = sendSQLCommand(sql)
newanswer = []
for temp in answer:
    newanswer.append({'Filing Date':str(temp['Filing Date']),'Cases':temp['Cases']})
df = pd.DataFrame(newanswer)
df['Filing Date'] = df['Filing Date'].astype(int)
df = df.loc[(df["Filing Date"] >= 1991)]
display(df.sort_values('Filing Date',ascending=True).head(30))
title = 'Criminal Cases in Non-Johnson County By Filing Date'
temp = df.sort_values(by=['Filing Date'],ascending=True).head(30).plot(kind="bar",x='Filing Date',xlabel = 'Filing Year',ylabel="Amount of Cases",title=title)
temp.figure.savefig("Graphs/" + title + ".png",bbox_inches="tight")

In [ ]:
#Charges By Disposition
sql = '''
SELECT Disposition,COUNT(*) as Dispositions FROM kansas2022.disposition_events WHERE `Event Type` = ' Disposition' GROUP BY Disposition 
UNION 
SELECT finding as Disposition,COUNT(*) as Dispositions FROM joco2022.charges GROUP BY finding'''
answer = sendSQLCommand(sql)
df = pd.DataFrame(answer)
display(df.sort_values('Dispositions',ascending=False).head(50))
newanswer = []
guilties = 0
dismisbypros = 0
for temp in answer:
    if(temp['Disposition'] != None and temp['Disposition'].upper() == 'GUILTY'):
        guilties += temp['Dispositions']
    elif(temp['Disposition'] != None and (temp['Disposition'].upper() == 'DISMISS BY PROS' or temp['Disposition'] == 'Dismissed by Prosecutor')):
        dismisbypros += temp['Dispositions']
    else:
        newanswer.append(temp)
newanswer.append({'Disposition':'Guilty','Dispositions':guilties})
newanswer.append({'Disposition':'Dismissed by Prosecution','Dispositions':dismisbypros}) 

df = pd.DataFrame(newanswer)
display(df.sort_values('Dispositions',ascending=False).head(10))
title = 'Top Ten Dispositions'
temp = df.sort_values(by=['Dispositions'],ascending=False).head(10).plot(kind="bar",x='Disposition',title=title,ylabel="Amount of Dispositions")
temp.figure.savefig("Graphs/" + title + ".png",bbox_inches="tight")